In [151]:
from flask import Flask
from flask import request
import werkzeug
import cv2
from PIL import Image
import mediapipe as mp
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Class containing Final Results for MediaPipe

In [152]:
class Results:
    
    cols = ["x","y","z","vis"]
    #TRAGULAR DATA
    trag_cols = ["x","y","z","vis"]
    trag_left_rows = ["left_ear","left_index"]
    trag_right_rows = ["right_ear","right_index"]
    tragular_left_coord = []
    tragular_right_coord = []
    
    #LUMBAR DATA
    lumbar_cols = ["x","y","z","vis"]
    lumbar_rows = ["left_index","right_index"]
    lumbar_neutral_coord = []
    lumbar_left_coord = []
    lumbar_right_coord = []
    
    #INTERMALLEOLAR DATA
    intermalleolar_rows = ["left_ankle","left_heel","left_foot_index",
                          "right_ankle","right_heel","right_foot_index"]
    intermalleolar_coord = []
    
    #CERVICAL DATA
    cervical_cols = ["x","y","z","vis","euler_angle"]
    cervical_left_coord = []
    cervical_neutral_coord = []
    cervical_right_coord = []
    
    

In [164]:
def mp_holistic_tragular2(side,file):
    
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_holistic = mp.solutions.holistic 
    holistic = mp_holistic.Holistic(static_image_mode=True,model_complexity=2,enable_segmentation=True,refine_face_landmarks=True)
    image = cv2.imread(file)
    results = holistic.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
        annotated_image,
        results.face_landmarks,
        mp_holistic.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.
        get_default_pose_landmarks_style())
    
    cv2.imwrite("holistic_tragular.png",annotated_image)
    
    if(side == 0):
        coordinate_list = [[results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].visibility],
                                       [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].x,
                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].y,
                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].z,
                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_INDEX].visibility]]
        Results.tragular_left_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_left_coord.index = Results.trag_left_rows
        print(Results.tragular_left_coord)
    else:
        coordinate_list = [[results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].x,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].y,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].z,
                                        results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].visibility],
                                       [results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].x,
                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].y,
                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].z,
                                       results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].visibility]]
        Results.tragular_right_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_right_coord.index = Results.trag_right_rows
        print(Results.tragular_right_coord)
    

# Functions for Measurements
    

In [165]:
#Function used to calculate the relevant tragular coordinates
def mp_pose_tragular(side,file):
    
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=6),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=5, circle_radius=5))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("tragular.png",image)
    
    if(side == 0):
        coordinate_list = [[results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].z,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].visibility],
                                       [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].x,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].y,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].z,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].visibility]]
        Results.tragular_left_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_left_coord.index = Results.trag_left_rows
        print(Results.tragular_left_coord)
    else:
        coordinate_list = [[results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].x,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].y,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].z,
                                        results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EAR].visibility],
                                       [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].x,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].y,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].z,
                                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].visibility]]
        Results.tragular_right_coord = pd.DataFrame(coordinate_list,columns=Results.trag_cols)
        Results.tragular_right_coord.index = Results.trag_right_rows
        print(Results.tragular_right_coord)
    

In [166]:
#Function used to calculate relevant lumbar coordinates
def mp_pose_lumbar(side,file):
    
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=6),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=5, circle_radius=5))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("lumbar.png",image)

    coordinate_list = [[results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_INDEX].visibility],
                           [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX].visibility]]

    
    if(side == -1):
        Results.lumbar_left_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_left_coord.index = Results.lumbar_rows
        print(Results.lumbar_left_coord)
    elif(side == 0):
        Results.lumbar_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_neutral_coord.index = Results.lumbar_rows
        print(Results.lumbar_neutral_coord)
    else:
        Results.lumbar_right_coord = pd.DataFrame(coordinate_list,columns=Results.lumbar_cols)
        Results.lumbar_right_coord.index = Results.lumbar_rows
        print(Results.lumbar_right_coord)
    
    
    

In [167]:
def mp_pose_intermalleolar(file):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=5, circle_radius=6),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=5, circle_radius=5))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("intermalleolar.png",image)
    
    coordinate_list = [[results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].visibility],
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HEEL].visibility],
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].visibility],
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].visibility],
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL].visibility],
                            [results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z,
                            results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].visibility]]
    Results.intermalleolar_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
    Results.intermalleolar_coord.index = Results.intermalleolar_rows
    

In [168]:
def mp_pose_cervical(file,side):
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode = True,model_complexity = 2, enable_segmentation=True,min_detection_confidence = 0.5)
    image = cv2.imread(file)
    results = pose.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=50, circle_radius=10),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
        thickness=50, circle_radius=10))
    mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imwrite("cervical.png",image)
    
    coordinate_list = [[results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z,
                       results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility]]
    
    if side == -1:
        Results.cervical_left_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_left_coord.index = ["left_nose_coordinates"]
    elif side == 0:
        Results.cervical_neutral_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_neutral_coord.index = ["neutral_nose_coordinates"]
    else:
        Results.cervical_right_coord = pd.DataFrame(coordinate_list,columns=Results.cols)
        Results.cervical_right_coord.index = ["right_nose_coordinates"]
    print(Results.cervical_left_coord)
    print(Results.cervical_neutral_coord)
    print(Results.cervical_right_coord)
    
    
    

# Server Code

In [ ]:
app = Flask(__name__)

@app.route("/")
def showHomePage():
    return "This is home page"

@app.route("/debug",methods=["POST"])
def debug():
    text = request.form["sample"]
    print(text)
    return "received"

####POSTING RELEVANT IMAGES TO SERVER###

@app.route("/tragularLeft",methods=["POST"])
def tragularLeft():
    imageFile = request.files['leftTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_tragular(0,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/tragularRight",methods=["POST"])
def tragularRight():
    imageFile = request.files['rightTragular']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_holistic_tragular2(1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarLeft",methods=["POST"])
def lumbarLeft():
    imageFile = request.files['leftLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_lumbar(-1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarNeutral",methods=["POST"])
def lumbarNeutral():
    imageFile = request.files['neutralLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_lumbar(0,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/lumbarRight",methods=["POST"])
def lumbarRight():
    imageFile = request.files['rightLumbar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_lumbar(1,filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/intermalleolar",methods=["POST"])
def intermalleolar():
    imageFile = request.files['intermalleolar']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_intermalleolar(filename)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalLeft",methods=["POST"])
def cervicalLeft():
    imageFile = request.files['leftCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,-1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalNeutral",methods=["POST"])
def cervicalNeutral():
    imageFile = request.files['neutralCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,0)
    print("\nSuccessful upload")
    return "Image uploaded successfully"

@app.route("/cervicalRight",methods=["POST"])
def cervicalRight():
    imageFile = request.files['rightCervical']
    filename = werkzeug.utils.secure_filename(imageFile.filename)
    print("\nReceived image file name: "+imageFile.filename)
    imageFile.save(filename)
    mp_pose_cervical(filename,1)
    print("\nSuccessful upload")
    return "Image uploaded successfully"
    

if __name__ == "__main__":
    app.run(host="0.0.0.0")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://138.38.166.67:5000/ (Press CTRL+C to quit)
138.38.214.43 - - [14/Jul/2022 18:53:08] "POST /debug HTTP/1.1" 200 -


TRAGULAR CONNECTED

Received image file name: right_tragular.png


138.38.214.43 - - [14/Jul/2022 18:53:20] "POST /tragularRight HTTP/1.1" 200 -


                    x         y         z       vis
right_ear    0.408433  0.373769 -0.862463  0.998809
right_index  0.422074  0.435429 -2.076323  0.959977

Successful upload


In [162]:
Results.cervical_left_coord

,x,y,z,vis
left_nose_coordinates,0.698279,0.582733,-0.833601,0.998341
